In [182]:
import pandas as pd
from pandas import json_normalize
import json
from json import loads, dumps
import pprint

In [183]:
# Create dataframes
df_replies = pd.read_csv('../data/twitter/tweets_isReply.csv', dtype={'id': 'object'}, low_memory=False)
df_tweets = pd.read_csv('../data/twitter/tweets_isTweet.csv', dtype={'id': 'object'}, low_memory=False)
df_nonQuotedTweets = df_tweets[df_tweets['quoted_tweet'].isna()]
df_quotedTweets = df_tweets[['id', 'text', 'quoted_tweet']].dropna(subset=['quoted_tweet'])

In [184]:
# Prepare quoted tweets
# Rename columns
df_quotedTweets.rename(columns={'id': 'tweet_id', 'text': 'tweet_text'}, inplace=True)

# Normalize quoted_tweet column
df_quotedTweets_normalized = pd.json_normalize(
    df_quotedTweets['quoted_tweet'].apply(json.loads),
    sep='_'
)

# Link both dataframes by index
df_quotedTweets_normalized.index = df_quotedTweets.index

# Rename columns
df_quotedTweets_normalized.columns = ['quoted_tweet_' + col for col in df_quotedTweets_normalized.columns]

# Concatenate the two dataframes
df_final = pd.concat([
    df_quotedTweets[['tweet_id', 'tweet_text']],
    df_quotedTweets_normalized[['quoted_tweet_id', 'quoted_tweet_text']]
], axis=1)

df_final.head()

,tweet_id,tweet_text,quoted_tweet_id,quoted_tweet_text
2,1917225430702240067,This is a big deal,1917223651625099407,"Last week, Treasury went live with its first a..."
4,1917103264417649121,Whoa,1917011279757066291,🚨THE INVISIBLE PUPPET MASTERS: AI'S DISTURBING...
5,1917099777327829386,"Next week, Grok 3.5 early beta release to Supe...",1917011847623987257,🚨GROK 3 SENDS USAGE SOARING – 10X SPIKE IN DOW...
6,1917071819003334728,It is an existential crisis!,1917059115417014780,A friendly reminder to make more babies!\n\n🇯🇵...
9,1917040536378335721,Starlink is trying out a service plan commitme...,1917029886432317947,$0 for the Standard Kit with 12-month resident...


In [59]:
visual = df_tweets.head().to_json(orient='split')
parsed = loads(visual)
parsed


'{\n    "columns": [\n        "type",\n        "id",\n        "url",\n        "twitterUrl",\n        "text",\n        "source",\n        "retweetCount",\n        "replyCount",\n        "likeCount",\n        "quoteCount",\n        "viewCount",\n        "createdAt",\n        "lang",\n        "bookmarkCount",\n        "isReply",\n        "inReplyToId",\n        "conversationId",\n        "inReplyToUserId",\n        "inReplyToUsername",\n        "author",\n        "extendedEntities",\n        "card",\n        "place",\n        "entities",\n        "quoted_tweet",\n        "retweeted_tweet"\n    ],\n    "index": [\n        0,\n        1,\n        2,\n        3,\n        4\n    ],\n    "data": [\n        [\n            "tweet",\n            1917726279195058338,\n            "https://x.com/elonmusk/status/1917726279195058338",\n            "https://twitter.com/elonmusk/status/1917726279195058338",\n            "https://t.co/U6tI9pdin6",\n            "Twitter for iPhone",\n            17246,\n

In [31]:
ent = df_replies['entities'].apply(json.loads)
ent.head()

0    {'user_mentions': [{'id_str': '18433784', 'ind...
1    {'user_mentions': [{'id_str': '151870962792763...
2    {'user_mentions': [{'id_str': '4914384040', 'i...
3    {'user_mentions': [{'id_str': '139388074369739...
4    {'user_mentions': [{'id_str': '57441414', 'ind...
Name: entities, dtype: object